In [112]:
import requests
import json
import copy

In [226]:
urlCreateDialOuts = 'https://callapi.gravitel.ru/api/v1/createdialout'

urlAmo = 'https://testcallapi.amocrm.ru'

urlAuthAmo = '/private/api/auth.php?type=json'
urlPipeAmo = '/api/v2/pipelines'
urlLeadAmo = '/api/v2/leads'

urlSipUri = '@test.gravitel.ru'

##"USER_LOGIN": "nd@gravitel.ru",
##"USER_HASH": "5e0db73b6bb4ddf46fcb4107090cffdcc5026961"

# Enter login & Hash

loginUserAmo = 'nd@gravitel.ru'#input("Enter LOGIN user: ")
hashUserAmo = '5e0db73b6bb4ddf46fcb4107090cffdcc5026961'#input("Enter HASH user: ")

# Setting parameters

headers = {'Content-Type': 'application/json'}
authKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo}

# Connect with amoCRM POST

authAmo = requests.post(urlAmo + urlAuthAmo, headers=headers, json=authKeyAmo).json()

# Список сотрудников

staff = [{'daniil': 2285464}, {'ilshat': 5906116}]

# check login & Hash

if 'error_code' in authAmo['response'].keys():
    print(authAmo['response']['error'])

## urlListLeadsAmo = '/api/v2/leads'
#print(authAmo['response'].keys())


In [114]:
# Получить все воронки и этапы, метод GET
'''
pipeKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo, 'id': 3139387}
pipelines = requests.get(urlAmo + urlPipeAmo, headers=headers, params=pipeKeyAmo).json()

if 'title' in pipelines.keys():
    print(pipelines['detail'])


'''
#print(pipelines)

"\npipeKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo, 'id': 3139387}\npipelines = requests.get(urlAmo + urlPipeAmo, headers=headers, params=pipeKeyAmo).json()\n\nif 'title' in pipelines.keys():\n    print(pipelines['detail'])\n\n\n"

In [227]:
# Получить все сделки, метод GET

leadKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo}
leads = requests.get(urlAmo + urlLeadAmo, headers=headers, params=leadKeyAmo).json()

# Проверка на ошибки

if 'title' in leads.keys():
    print(leads['detail'])

# Делаем из словаря список словарей
    
leads = leads['_embedded']['items']



for i, elem in enumerate(leads):
    
    # Удаляем сделки, если этап не 32121847 (НЕДОЗВОН)
    if leads[i]['status_id'] != 32121847:
        del leads[i]

# Создаем словарь с нужными ключами

leadsClear = copy.deepcopy(leads)

for i, elem in enumerate(leads):
    for key in leads[i]:
        if key not in ('id', 'responsible_user_id', 'main_contact'):
            leadsClear[i].pop(key)
    if leadsClear[i]['main_contact'] == {}: # Проверяем есть ли контакт у сделки, если нет, то удаляем сделку из БД.
        del leadsClear[i]
        
# Достаем номер из Контакта сделки

for i, elem in enumerate(leadsClear):
    urlContactAmo = leadsClear[i]['main_contact']['_links']['self']['href']
    contact = requests.get(urlAmo + urlContactAmo, headers=headers, params=leadKeyAmo).json()
    leadsClear[i]['main_contact'] = int(contact['_embedded']['items'][0]['custom_fields'][0]['values'][0]['value'])
    leadsClear[i]['count'] = 3

# Меняем id из АМО на логин из ВАТС

for i, elem in enumerate(leadsClear):
    for j, elem in enumerate(staff):
        for key in staff[j]:
            if leadsClear[i]['responsible_user_id'] == staff[j][key]:
                leadsClear[i]['responsible_user_id'] = key
json.dump(leadsClear,open("leadsClear.txt","w"))





leadsClear

[{'id': 1472181,
  'responsible_user_id': 'daniil',
  'main_contact': 79852721228,
  'count': 3},
 {'id': 2036907,
  'responsible_user_id': 'daniil',
  'main_contact': 79096546816,
  'count': 3},
 {'id': 2039071,
  'responsible_user_id': 'ilshat',
  'main_contact': 79852721228,
  'count': 3},
 {'id': 2036873,
  'responsible_user_id': 'ilshat',
  'main_contact': 74952302222,
  'count': 3}]

In [195]:
# Запись в файл
#json.dump(leads,open("leads.txt","w"))

In [ ]:
'''
for i, elem in enumerate(leads):
    for leadsClear_values in iter(leadsClear.items()):
        if leads[i]['responsible_user_id'] == leadsClear[leadsClear_values]: 
            print ('Ok', 'responsible_user_id', leadsClear_values)
        else: 
            print ('Not', leads_values, leadsClear_values)
'''